In [8]:
import torch
import csv
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from tqdm import tqdm

In [4]:
def get_encode_data(tokenizer, sentences, max_length=128):
    encoded_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=max_length)
    input_ids = torch.tensor(encoded_inputs['input_ids'])
    attention_masks = torch.tensor(encoded_inputs['attention_mask'])

    return (input_ids, attention_masks)

In [5]:
model_name = 'klue/roberta-base'
b_model=128
length=256
code_file = '/home/osung/work/lm_classification/klue_roberta-base_ad_train.tsv_ad_code.csv'

In [6]:
pretrained_file = '/home/osung/work/lm_classification/klue_roberta-base_ad_total.tsv_ad_b128_e10_ml256_20240814_135702.pth'
num_labels = 2

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('available device: ', device)
else:
    device = torch.device("cpu")
    print('available device: ', device)

available device:  cuda


In [120]:
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False,)

In [121]:
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = 'left'

In [122]:
pretrained_model_config = AutoConfig.from_pretrained(model_name)

In [123]:
pretrained_model_config.num_labels = num_labels

In [124]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config=pretrained_model_config,
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#model.config.pad_token_id = model.config.eos_token_id

In [125]:
model = torch.nn.DataParallel(model)

In [126]:
model = model.to(device)

In [22]:
codes = {}
with open(code_file, 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        key, value = row
        codes[key] = int(value)

In [23]:
codes[True] = codes['True']
codes[False] = codes['False']

In [127]:
codes

{'False': 0, 'True': 1, True: 1, False: 0}

In [41]:
print(model)

DataParallel(
  (module): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): 

In [128]:
loaded_state_dict = torch.load(pretrained_file)

In [129]:
model.load_state_dict(loaded_state_dict)

<All keys matched successfully>

In [130]:
model.eval()

DataParallel(
  (module): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): 

In [9]:
test_df = pd.read_csv('/home/osung/data/korean/modu/json/combined_news.tsv', sep='\t')

In [10]:
test_df.text[:10]

0    "대통령, 시장 방문만 하지 말고 실천해달라" 2008년의 마지막 새벽, 언론의 카...
1    진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신...
2    "소의 해, 정치인들 싸움 좀 그만하시죠" 지난해 12월 31일 대구국채보상기념운동...
3    'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...
4    산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...
5    "방송장악은 군사독재로 회귀하는 것" 한나라당이 추진 중인 언론법과 관련해 언론노조...
6    '만수'의 해에 국민들은 '명박'했다소의 해에는 뚜벅뚜벅 우보로 가자 지나간 한 해...
7    지리산 '2MB OUT 소망 띄우기', 경찰 저지로 무산 지리산 삼신봉에서 새해 첫...
8    박희태 "신라시대 만파식적은 어디 있는가?" 청와대의 바람과 달리 소위 'MB법안'...
9    모두에게 '윈-윈 게임'이 되는 세상, 아름다운 동행 21세기는 IQ(지능지수, I...
Name: text, dtype: object

In [15]:
test_df.text[50]

'그녀의 번뇌와 선택이 시사하는 것 북에서도 단정 수립 기도를... 이강국은 저간의 사정을 이야기했다. 그는 민족의 분단을 막는 일을 자기의 소명으로 알고 있었다. 남한에서 단독정부 수립을 비판하던 그는 미 군정청의 수배를 받게 되자 북한으로 넘어갔다. 북한에 간 그는 소스라치게 놀랐다. 단정 수립 기도는 남한만 하고 있던 것이 아니기 때문이었다. 북한의 김일성도 소련의 지시를 받아 단독 정부 수립을 위해 노골적으로 움직이고 있었다. 이강국은 남로당의 간부였다. 김일성의 북로당은 노동법 전문가이고 외국어에 능통한 그에게 외무국장의 직책을 맡겼다. 이강국은 북로당의 단정 수립 움직임에 제동을 걸었다. 또한 그는 북로당의 여러 정책에 문제를 제기했다. 김일성의 부하들은 그에게 부르주아의 때가 남아 있다고 생각했다. 그는 남한에서 미군정에 소련을 무시하지 말라고 말했던 것처럼 북로당에는 미국을 배척하지 말라고 경고했다. 그러다가 그는 당의 명령에 불복했다는 이유로 직위 해제되었다. 당은 그에게 3개월 동안 감옥에서 자아비판을 하라고 명령했다. 김일성은 감옥에서 나온 그를 다시 남한으로 보내고 싶었다. 그래서 그에게 남로당 재건의 책임을 맡겼다. 박헌영의 남로당은 이미 미군정과 이승만의 탄압으로 궤멸 상태에 있었다. "나는 남조선 정부와 미군 정보기관에서 현상금을 걸고 수배한 사람이야. 내가 수임이를 함부로 만나게 되면 수임이까지 프락치로 몰리게 되어 있어. 그래도 한 번 수임이를 보고 싶었어. 딱 한 번만 보려고 했던 거지. 이제 수임이를 보았고 행복하게 사는 것을 알았으니 됐어." 이강국은 피곤한 듯 발을 뻗으며 담배를 피워 물었다. 밤이 깊어지고 있었다. 괘종시계가 열 번을 쳤는지 열한 번을 쳤는지 그녀는 알 수 없었다. 이강국은 몸을 일으키며 말했다. "그럼 이제 일어나지." "이 밤에 어딜요?" "바래다주마. 오늘 이후로 우리는 만나면 안 돼." 김수임은 왈칵 눈물이 솟았다. "저는 행복하지 않아요. 베어드와 살림을 차린 것은 제가 정말 원했던 일이 아니었어

In [52]:
encode_data = get_encode_data(tokenizer, [test_df.text[0]], max_length=256)

In [53]:
encode_data

(tensor([[    0,     6,  3698,    16,  3648,  4110,  2154,  1889,  2118,  1041,
           2088,  4980,  2097,  2448,  2181,     6,  4898,  2440,  2079,  4178,
           5566,    16,  4163,  2079,  5677,  2259,  3671,  7915,  2138,  9991,
           2062,    18, 19817,   886,  4132,  2729,  2532,  5698,  2052,  9691,
           2651,  3906, 10859,  2121,  2069, 24402,  4833,  3712,  2031,  2079,
           4175,  2069,  5037,  2088,  1513,  2359,  2414,   575,    18,   555,
           2073,  4808,    16,  3906,  1120,  3991, 27135,  5946,  2031,  2073,
           3674,  4098,  2522, 18073,  2090,  1283, 27135,  1237,  2079, 16825,
           2530,  2145,  3713,  2170,  3618, 29126,  2170, 16020,  1643, 19329,
           2470,  1237,  2069,  3796,  2116,  2088,  1513,  2359,  2062,    18,
           1504,  2031,  2079,  4243,  2138,   881,  2015,  3627,  3814,  5566,
             23,  2067,  4892,  2079,  4408,  2069,  1428,  2259,  3671, 14597,
           2328, 18861, 27854,  2119,  2

In [55]:
output = model(encode_data[0], encode_data[1])

In [56]:
output

SequenceClassifierOutput(loss=<generator object gather.<locals>.gather_map.<locals>.<genexpr> at 0x7fde4ecf9f20>, logits=tensor([[-4.2776,  4.4596]], device='cuda:0', grad_fn=<GatherBackward>), hidden_states=None, attentions=None)

In [58]:
_, predicted = torch.max(output.logits, dim=1)

In [80]:
predicted = predicted.to(torch.device('cuda'))

In [81]:
predicted = predicted.item()

In [82]:
predicted

1

In [60]:
codes

{'False': 0, 'True': 1, True: 1, False: 0}

In [16]:
ad_df = test_df[test_df['text'].apply(lambda x: all(keyword in x for keyword in ['아파트', '분양', '건설', '분양가', '모델하우스']))]

In [17]:
ad_df.text[:50]

1406      고분양가 그대로 두고 미분양 해소! 바보 아냐? "건설사들이 '같이 죽든지 아니면 ...
2245      아파트 계약자들과 시공사 만남이 '화기애애'? 건설업체가 변하고 있다. 장기침체로 ...
2576      도심 재개발 원주민 정든 마을 내쫓길 처지 재개발 보상가 문제로 일부 주민들의 거센...
9456      "서해 조망권이라더니 민둥산 조망권이었나?" "강남에 지은 곳보다 건축비가 더 들었...
14437     대법, 분양률 높이려 과장 광고한 건설사 제동 신규 아파트 인근에 전철역이 들어선다...
17327     "값 비싼 불량 아파트... 살고 싶지 않다" 아파트 모델하우스와 홍보책자만큼 한국...
62586     소형 평형(구25평형) 아파트 투자가 대세!!! 우미 건설은 영종 하늘 도시 내에 ...
62596     청약통장 필요 없는 25평형 아파트 특별분양 “영종하늘도시 우미린” 우미 건설은 영...
62609     1500만원대로 2억대 신도시 아파트 잡으세요! -양도세가 얼마인데 100% 면제 ...
62666     부동산 재테크 열기, 주목할 만한 곳은? 국내 실물경기 회복에 따라 올해 하반기 이...
65401     프리미엄이란 이런곳! 제주도 고급형 빌라 '센텀 일마레' 절찬리 분양중! 제주도는 ...
82884     세종시 첫마을 청약에 '떴다방' 등장 세종시 첫마을 '퍼스트프라임' 청약 접수가 9...
90738     울산·부산 '포스코 더샵' 이유있는 인기 포스코건설(사장 정동화)이 울산 '문수산 ...
109165    현대건설, '광교 힐스테이트 레이크' 14일 오픈 현대건설이 오는 14일 경기도 수...
112774    '송도 더샵 마스터뷰' 견본주택 3일간 2만5천여명 방문 GCF 사무국 유치 발표 ...
113919    구미 임은 삼도뷰엔빌W '분양 활기'…부동산 경기 청신호? 경북 구미 임은 삼도뷰엔...
123700    한화건설, 상암 DMC 바로 앞 1억원대 '오피스텔' 

In [20]:
test_df.loc[1406].text

'고분양가 그대로 두고 미분양 해소! 바보 아냐? "건설사들이 \'같이 죽든지 아니면 미분양을 해소시켜 달라\'고 하니 정부에서 나선 거다. 그런데 건설사가 \'(미분양) 책임 없다\'며 비싼 분양가를 안 내리니 미분양 해소가 되겠느냐." 지난 18일 경기 용인시 수지구 성복동에서 만난 공인중개사 김정수(가명)씨가 한숨 쉬듯 말했다. 지난 2월 12일 정부가 미분양 해소를 위해 내놓은 양도소득세 면제·감면 조치의 효과에 대해 묻자 돌아온 답이었다. 김씨의 공인중개사무소 인근에는 2·12 조치의 대표적인 수혜단지로 언론에 보도된 \'용인 성복 힐스테이트\'가 있다. 이 아파트를 두고 그는 "완전히 \'1+1\'"이라며 "힐스테이트 분양가격으로 인근 아파트 2채를 살 수 있다, 누가 사겠느냐?"고 지적했다. 언론의 섣부른 보도와는 달리 고분양가 탓에 미분양 해소라는 정책적 효과를 기대할 수 없다는 게 김씨의 생각이다. 또한 그는 양도세 감면·면제 혜택을 보지 못하는 계약자들이 아파트 분양권을 싸게 내놓아, 미분양 해소는 더욱 어렵다고 덧붙였다. 이는 2·12 조치의 다른 수혜단지의 사정도 마찬가지였다. 모델하우스 북적? 빈 책상에 앉은 분양 상담사들 이날 오전에 찾은 경기 성남시 분당구 구미동 \'용인 성복 힐스테이트\' 모델하우스는 적막감이 감돌았다. 상담사 3명이 빈 책상에 앉아 방문객을 기다리고 있었다. 이곳의 분위기는 "용인 지역 미분양 아파트는 앞으로 5년간 양도세가 면제되기 때문에, 2·12 조치 발표 이후 방문객과 문의가 폭주하면서 미분양이 빠르게 소진되고 있다"는 일부 언론 보도와는 거리가 멀었다. 반면 현대건설 분양팀 관계자는 "2·12 조치 이후 하루 10명이던 방문객이 200여 명으로 늘었다"며 "가계약하겠다는 40~50명을 확보했다"고 말했다. 현재 모두 2157세대인 이 아파트단지의 미분양 물량은 700여 세대다. 이 관계자는 이어 "현재 양도세 감면 조치가 어느 정도 효과가 있을지는 지켜봐야 한다"며 "경기침체가 가속화되고 있는 상황에서 

In [83]:
def predict_ad(text, model, tokenizer) :
    encode_data = get_encode_data(tokenizer, [text], max_length=256)
    output = model(encode_data[0], encode_data[1])
    _, predicted = torch.max(output.logits, dim=1)
    predicted = predicted.item()
    
    if predicted : 
        return True
    else :
        return False

In [169]:
predict_ad(test_df.text[282643], model, tokenizer)

True